### 정확도가 향상된 이미지 분류기 만들기

이전주차보다 신경망의 정확도를 향상시키는 것을 목표로 한다. 신경망을 정의하는 부분에서 변화를 주어 여러가지 방법으로 접근해본다.

이전 주차에 사용했던 CIFAR-10 데이터셋을 사용한다. CIFAR-10에 포함된 이미지의 크기는 3x32x32로, 이는 32x32 픽셀 크기의 이미지가 3개 채널(channel)의 색상로 이뤄져 있다는 것을 의미한다.

다음과 같은 단계로 학습한다.

1. torchvision 을 사용하여 CIFAR-10의 학습(training), 평가(test) 데이터셋을 로드하여 정규화(nomarlizing)한다.

2. 합성곱 신경망(Convolution Neural Network)을 정의한다.

3. 손실 함수를 정의한다.

4. 학습용 데이터를 사용하여 신경망을 학습시킨다.

5. 시험용 데이터를 사용하여 신경망을 검사한다.


###  1단계 : CIFAR-10 로딩 및 정규화

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### 2~4 단계

In [13]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # First 2D convolutional layer, taking in 3 input channels (image)
        # 6 output channels, a square kernel size of 5
        self.conv1 = nn.Conv2d(3, 6, 5)
        
        # Max pooling over a (2, 2) mask
        self.pool = nn.MaxPool2d(2, 2)
        
        # Second 2D convolutional layer, taking in 6 input layers,
        # 16 output channels, a square kernel size of 5
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 256) # 5*5 from image dimension
        self.fc2 = nn.Linear(256, 84)
        self.fc3 = nn.Linear(84, 10)
    
    # x represents our data
    def forward(self, x):
        # Pass data through conv1
        # Use the rectified-linear activation function over x
        # Run max pooling over x
        x = self.pool(F.relu(self.conv1(x)))
        
        # Pass data through conv1
        # Use the rectified-linear activation function over x
        # Run max pooling over x
        x = self.pool(F.relu(self.conv2(x)))
        
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        
        return x


net = Net()

criterion = nn.CrossEntropyLoss()
learning_rate = 1e-4
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

for t in range(2):   # 데이터셋을 수차례 반복

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data

        # 순전파
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        
        #변화도를 0으로 만들고, 역전파 단계를 수행 후 가중치 갱신
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()

        # 통계를 출력
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (t + 1, i + 1, running_loss / 2000))
            running_loss = 0.0        
        
    
        
print('Finished Training')

[1,  2000] loss: 1.990
[1,  4000] loss: 1.748
[1,  6000] loss: 1.666
[1,  8000] loss: 1.620
[1, 10000] loss: 1.572
[1, 12000] loss: 1.569
[2,  2000] loss: 1.492
[2,  4000] loss: 1.485
[2,  6000] loss: 1.460
[2,  8000] loss: 1.428
[2, 10000] loss: 1.411
[2, 12000] loss: 1.411
Finished Training


### 5. 시험용 데이터로 신경망 검사하기

In [14]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 50 %
